## Imports

In [1]:
import random
import re
from functools import partial

In [2]:
from lasagne.layers import Layer, InputLayer, EmbeddingLayer
from lasagne import nonlinearities
from lasagne.updates import rmsprop

In [3]:
from spaghetto.model import RNN
from spaghetto.utils import TokenEncoder
from spaghetto.recurrent import GRULayer
from spaghetto.recurrent import RNNDenseLayer

/home/bbossan_dev/anaconda/envs/spaghetto/lib/python2.7/site-packages/Lasagne-0.1.dev0-py2.7.egg/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


## Create artificial data

Data is of the type _13+45=58_

In [4]:
X = ["{}+{}={}".format(a, b, a + b) for a in range(100) for b in range(100)]

In [5]:
encoder = TokenEncoder().fit(X)

In [6]:
random.shuffle(X)

In [7]:
Xtrain, Xtest = X[:8000], X[8000:]

## Initialize net

In [8]:
embedding_size = 16
num_units = 32
learning_rate = 1e-2

In [9]:
layers = [
    (InputLayer, {}),
    (EmbeddingLayer, {'output_size': embedding_size}),
    (GRULayer, {'grad_clipping': 5., 'num_units': num_units}),
    (RNNDenseLayer, {'nonlinearity': nonlinearities.identity}),
]

In [12]:
rnn = RNN(
    layers,
    encoder=encoder,
    verbose=1,
    updater=partial(rmsprop, learning_rate=learning_rate),
    max_epochs=50,
    batch_size=128,
    max_len_line=12,
)

## Train

In [13]:
rnn.fit(Xtrain)

/home/bbossan_dev/anaconda/envs/spaghetto/lib/python2.7/site-packages/Lasagne-0.1.dev0-py2.7.egg/lasagne/layers/helper.py:69: UserWarning: get_all_layers() has been changed to return layers in topological order. The former implementation is still available as get_all_layers_old(), but will be removed before the first release of Lasagne. To ignore this warning, use `warnings.filterwarnings('ignore', '.*topo.*')`.
  warnings.warn("get_all_layers() has been changed to return layers in "
/home/bbossan_dev/anaconda/envs/spaghetto/lib/python2.7/site-packages/theano/scan_module/scan.py:1017: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)


 - Compiling functions ...
   ... finished compilation.

  epoch    train perpl.    valid perpl.    train/valid  duration    sample
-------  --------------  --------------  -------------  ----------  ----------
      1         7.38919         7.18130        1.02894  1.98s       81+41=67
      2         4.29909         4.29150        1.00178  2.13s       57+7=98
      3         4.13031         4.12520        1.00123  2.17s       69+70=124
      4         4.01943         4.01490        1.00113  2.09s       24+72=95
      5         3.93380         3.93200        1.00045  2.14s       82+70=187
      6         3.86750         3.86780        0.99992  2.19s       6+73=89
      7         3.80652         3.80830        0.99954  2.13s       56+29=84
      8         3.75030         3.75420        0.99897  2.06s       28+44=64
      9         3.70113         3.70880        0.99793  2.06s       26+4=48
     10         3.66024         3.67130        0.99698  2.07s       30+51=93
     11         3.62

/home/bbossan_dev/anaconda/envs/spaghetto/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:133: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


## Sample from unseen data

In [17]:
print("Input | expected | predicted | error")
print("-" * 38)
for xi in Xtest[:20]:
    x, y = xi.split('=')
    pred = rnn.sample(1, primer=x)[0].split('=')[1]
    pred = re.sub("\\\n", "", pred)

    print("{:>5} | {:>8} | {:>9} | {:>3}."
          "".format(x, y, pred, int(y) - int(pred)))

Input | expected | predicted | error
--------------------------------------
48+38 |       86 |        97 | -11.
 6+25 |       31 |        41 | -10.
49+45 |       94 |       100 |  -6.
33+18 |       51 |        58 |  -7.
40+15 |       55 |        60 |  -5.
58+38 |       96 |        94 |   2.
31+95 |      126 |       137 | -11.
 1+72 |       73 |        75 |  -2.
90+26 |      116 |       127 | -11.
 4+27 |       31 |        30 |   1.
19+29 |       48 |        57 |  -9.
78+45 |      123 |       125 |  -2.
77+31 |      108 |       118 | -10.
68+57 |      125 |       128 |  -3.
32+21 |       53 |        60 |  -7.
84+53 |      137 |       137 |   0.
29+76 |      105 |       117 | -12.
47+39 |       86 |        90 |  -4.
86+16 |      102 |        85 |  17.
10+74 |       84 |        88 |  -4.
